# Машинное обучение для текстов

## Проект для викишоп

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

**Цель проекта:** обучить модель классифицировать комментарии на негативные и позитивные.

**Входные данные:** набор данных с разметкой о токсичности правок

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from random import randint
from sklearn.metrics import f1_score
from tqdm import tqdm
from scipy import sparse
from torch.nn.utils.rnn import pad_sequence
import warnings
random_state=12345

### Загрузка и подготовка данных

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
try:
    df_comments = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/toxic_comments.csv")
except:    
    df_comments = pd.read_csv("/datasets/toxic_comments.csv") 

In [ ]:
df_comments = df_comments.sample(n=50000, random_state=12345)

In [ ]:
df_comments.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 146790 to 110088
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    50000 non-null  object
 1   toxic   50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ MB


In [ ]:
df_comments.head()

,text,toxic
146790,Ahh shut the fuck up you douchebag sand nigger...,1
2941,"""\n\nREPLY: There is no such thing as Texas Co...",0
115087,"Reply\nHey, you could at least mention Jasenov...",0
48830,"Thats fine, there is no deadline ) chi?",0
136034,"""\n\nDYK nomination of Mustarabim\n Hello! You...",0


In [ ]:
df_comments.isna().sum()

text     0
toxic    0
dtype: int64

In [ ]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
warnings.filterwarnings('ignore') 

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
#Лемматизируем текст и избавимся от ненужных символов
def lemmatize(text):
    text = text.lower()
    text = re.sub(r'[^a-z]', ' ', text)
    m = WordNetLemmatizer()
    lemm_list = [m.lemmatize(word) for word in nltk.word_tokenize(text)]
    lemm_text = " ".join(lemm_list)   
    return lemm_text

In [ ]:
tqdm.pandas()
df_comments['text'] = df_comments['text'].progress_apply(lemmatize)

100%|██████████| 50000/50000 [00:43<00:00, 1136.69it/s]


In [ ]:
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))
warnings.filterwarnings('ignore') 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
features = df_comments.drop('toxic', axis=1)
target = df_comments['toxic']

In [ ]:
#Разделим выборку на обучающую и тестовую в соотношении 3:1
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.25, random_state=random_state, stratify=target)

In [ ]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf_train = count_tf_idf.fit_transform(x_train['text'].values)

In [ ]:
tf_idf_test = count_tf_idf.transform(x_test['text'].values)

In [ ]:
x_train = pd.DataFrame.sparse.from_spmatrix(tf_idf_train)
x_test = pd.DataFrame.sparse.from_spmatrix(tf_idf_test)

### Обучение моделей

In [ ]:
#Посмотрим на соотношение классов
print("Количество положительных классов в обучающей выборке:", len(y_train[y_train==1]))
print("Количество отрицательных классов в обучающей выборке:", len(y_train[y_train==0]))

Количество положительных классов в обучающей выборке: 3810
Количество отрицательных классов в обучающей выборке: 33690


<div class="alert alert-block alert-info">
Баланс классов присутствует, когда соотношение классов в выборке примерно 1:1. В нашем случае можно наблюдать дисбаланс. 
</div>

<div class="alert alert-block alert-info">
Обучим несколько моделей: модель логистической регрессии, градиентного бустинга и случайного леса. Подберём параметры по GridSearchCV и выберем наилучшую модель по f1_score на валидационной выборке.

In [ ]:
#Напишем функцию для подбора параметров модели градиентного бустинга
def select_params_boost(x, y):
    model_boost = CatBoostClassifier(task_type='GPU')
    parameters = {
        'random_state': [random_state],
        'n_estimators': [randint(1,50)],
        'learning_rate': [0.03, 0.1],
        'max_depth': [randint(1,16)]
                }
    grid = GridSearchCV(model_boost, parameters, verbose=False, scoring='f1')
    grid.fit(x, y)
    return grid.best_params_, grid.best_score_

In [ ]:
print("Параметры модели градиентного бустинга для обучения и f1_score :", select_params_boost(x_train, y_train))

0:	learn: 0.6628044	total: 58.5ms	remaining: 1.93s
1:	learn: 0.6332805	total: 119ms	remaining: 1.9s
2:	learn: 0.6071533	total: 174ms	remaining: 1.8s
3:	learn: 0.5833421	total: 234ms	remaining: 1.76s
4:	learn: 0.5600686	total: 279ms	remaining: 1.62s
5:	learn: 0.5386208	total: 316ms	remaining: 1.47s
6:	learn: 0.5186868	total: 363ms	remaining: 1.4s
7:	learn: 0.4996213	total: 406ms	remaining: 1.32s
8:	learn: 0.4824112	total: 443ms	remaining: 1.23s
9:	learn: 0.4662244	total: 484ms	remaining: 1.16s
10:	learn: 0.4514375	total: 518ms	remaining: 1.08s
11:	learn: 0.4371205	total: 554ms	remaining: 1.01s
12:	learn: 0.4242743	total: 586ms	remaining: 947ms
13:	learn: 0.4134659	total: 617ms	remaining: 882ms
14:	learn: 0.4030577	total: 650ms	remaining: 823ms
15:	learn: 0.3923186	total: 689ms	remaining: 775ms
16:	learn: 0.3830049	total: 720ms	remaining: 720ms
17:	learn: 0.3745529	total: 760ms	remaining: 675ms
18:	learn: 0.3661868	total: 793ms	remaining: 626ms
19:	learn: 0.3578073	total: 827ms	remaining

In [ ]:
print("Процент объектов положительного класса:", round((len(y_train[y_train==1]) / len(y_train)) * 100, 1))
print("Процент объектов отрицательного класса:", round((len(y_train[y_train==0]) / len(y_train)) * 100, 1))

Процент объектов положительного класса: 10.2
Процент объектов отрицательного класса: 89.8


In [ ]:
#Модель градиентного бустинга с подобранными параметрами, с учетом дисбаланса проставим коэффициенты для параметра class_weigths (выборка из 10000 объектов)
model_boost = CatBoostClassifier(random_state=random_state, n_estimators=36, learning_rate=0.1, max_depth=5, class_weights=[0.9, 0.1], task_type='GPU')

In [ ]:
#Напишем функцию для подбора параметров модели логистической регрессии
def select_params_regression(x, y):
    model = LogisticRegression()
    parameters = {
        'random_state': [random_state],
        'penalty': ['l1', 'l2'],
        'C': [randint(1,50)]
    }
    grid = GridSearchCV(model, parameters, verbose=False, scoring='f1')
    grid.fit(x, y)
    return grid.best_params_, grid.best_score_
warnings.filterwarnings('ignore') 

In [ ]:
print("Параметры модели логистической регрессии и f1:", select_params_regression(x_train, y_train))

Параметры модели логистической регрессии и f1: ({'C': 23, 'penalty': 'l2', 'random_state': 12345}, 0.7347393328268119)


In [ ]:
#Модель логистической регрессии с подобранными параметрами и сбалансированными классами
model_regression = LogisticRegression(random_state=random_state, penalty='l2', C=9, class_weight='balanced')

In [ ]:
#Напишем функцию для подбора параметров модели случайного леса
def select_params_random_forest(x, y):
    model = RandomForestClassifier()
    parameters = {
        'random_state': [random_state],
        'max_depth': [randint(1,16)],
        'n_estimators': [randint(1,100)]
    }
    grid = GridSearchCV(model, parameters, verbose=False, scoring='f1')
    grid.fit(x, y)
    return grid.best_params_, grid.best_score_

In [ ]:
print("Параметры модели случайного леса для обучения и f1_score:", select_params_random_forest(x_train, y_train))

Параметры модели случайного леса для обучения и f1_score: ({'max_depth': 1, 'n_estimators': 86, 'random_state': 12345}, 0.0)


In [ ]:
model_forest = RandomForestClassifier(random_state=random_state, max_depth=4, n_estimators=31, class_weight='balanced')

<div class="alert alert-block alert-info">
Таким образом, для предсказания негативных и позитивных комментариев берем модель логистической регрессии, так как по gridsearch метрика f1 показала наилучшее качество.

### Проверка на тестовой выборке, выводы 

In [ ]:
model_regression.fit(x_train, y_train)
predictions = model_regression.predict(x_test)
print("f1:", f1_score(y_test, predictions))

f1: 0.7510580992689495


<div class="alert alert-block alert-info">

**Вывод:** для предсказания токсичности комментариев, исходные данные были преобразованы, текст разбит на токены, а также лемматизирован и очищен от ненужных символов. Тексты были преобразованы в вектора при помощи TfIdfVectorizer.
Было обучено 3 вида моделей: модель градиентного бустинга, логистическая регрессия, случайный лес. Лучшее качество по метрике f1 показала модель логистической регрессии. Она и была использована с подобранными параметрами на тестовой выборке. Значение метрики f1 на тестовой выборке-0,75. Данная модель предлагается заказчику для предсказания.

## BERT

###Подготовка данных

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import transformers
from tqdm import notebook

In [ ]:
# инициализируем модель и токенизатор
model_name = "unitary/toxic-bert" 
model = transformers.AutoModel.from_pretrained(model_name)
tokenizer = transformers.BertTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# инициализируем токенизатор
tokenizer = transformers.BertTokenizer.from_pretrained("unitary/toxic-bert")

In [ ]:
tqdm.pandas()

tokenized = df_comments['text'].progress_apply(
lambda x: tokenizer.encode(x, max_length=512, truncation=True, add_special_tokens=True)) #обрежет под нужное кол-во

padded = pad_sequence([torch.as_tensor(seq) for seq in tokenized], batch_first=True) #добьет нулями  

attention_mask = padded > 0
attention_mask = attention_mask.type(torch.int) #Тут можно сделать, то как было в теории

100%|██████████| 50000/50000 [01:44<00:00, 479.93it/s]


In [ ]:
input_ids = np.asarray(padded)
attention_mask = np.asarray(attention_mask)

In [ ]:
%%time
from tqdm import notebook
batch_size = 2 # для примера возьмем такой батч, где будет всего две строки датасета
embeddings = [] 
for i in notebook.tqdm(range(input_ids.shape[0] // batch_size)):
        batch = torch.LongTensor(input_ids[batch_size*i:batch_size*(i+1)]).cuda() # закидываем тензор на GPU
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).cuda()

        with torch.no_grad():
            model.cuda()
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)

        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy()) # перевод обратно на проц, чтобы в нумпай кинуть
        del batch
        del attention_mask_batch
        del batch_embeddings

features = np.concatenate(embeddings) 

  0%|          | 0/25000 [00:00<?, ?it/s]

CPU times: user 33min 35s, sys: 12.3 s, total: 33min 47s
Wall time: 33min 50s


In [ ]:
features = pd.DataFrame(features)

###Обучение моделей

In [ ]:
#Разделим новые признаки на обучающую и тестовую выборку
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.25, random_state=random_state)

In [ ]:
#Подберем параметры для моделей градиентного бустинга, логистической регрессии и случайного леса на новых признаках (эмбедингах)
print("Параметры модели градиентного бустинга и f1_score на новых данных:", select_params_boost(x_train, y_train))

0:	learn: 0.6150859	total: 12.5ms	remaining: 87.7ms
1:	learn: 0.5520915	total: 23.3ms	remaining: 69.9ms
2:	learn: 0.4995922	total: 34.1ms	remaining: 56.9ms
3:	learn: 0.4522405	total: 44.8ms	remaining: 44.8ms
4:	learn: 0.4121513	total: 55.8ms	remaining: 33.5ms
5:	learn: 0.3716802	total: 66.6ms	remaining: 22.2ms
6:	learn: 0.3338902	total: 77.5ms	remaining: 11.1ms
7:	learn: 0.3047810	total: 89ms	remaining: 0us
0:	learn: 0.6223605	total: 11.9ms	remaining: 83.5ms
1:	learn: 0.5588874	total: 23.1ms	remaining: 69.2ms
2:	learn: 0.4923338	total: 34ms	remaining: 56.6ms
3:	learn: 0.4436842	total: 44.9ms	remaining: 44.9ms
4:	learn: 0.3961308	total: 56.4ms	remaining: 33.8ms
5:	learn: 0.3574005	total: 67.2ms	remaining: 22.4ms
6:	learn: 0.3248592	total: 78.2ms	remaining: 11.2ms
7:	learn: 0.2956247	total: 95.3ms	remaining: 0us
0:	learn: 0.6177136	total: 12.3ms	remaining: 86.3ms
1:	learn: 0.5568744	total: 23.1ms	remaining: 69.2ms
2:	learn: 0.4996050	total: 33.8ms	remaining: 56.4ms
3:	learn: 0.4548749	to

In [ ]:
print("Параметры модели логистической регрессии и f1_score на новых данных:", select_params_regression(x_train, y_train))

Параметры модели логистической регрессии и f1_score на новых данных: ({'C': 47, 'penalty': 'l2', 'random_state': 12345}, 0.9040604634602477)


In [ ]:
print("Параметры модели случайного леса и f1_score на новых данных:", select_params_random_forest(x_train, y_train))

Параметры модели случайного леса и f1_score на новых данных: ({'max_depth': 14, 'n_estimators': 43, 'random_state': 12345}, 0.9083006285369534)


<div class="alert alert-block alert-info">

**Комментарий студента:**

На новых признаках наилучшее качество модели по метрике f1 - модели градиентного бустинга, обе другие модели также показали высокое качество. Обучим эту модель с корректировкой дисбаланса классов и посчитаем качество на тестовой выборке.

In [ ]:
model = CatBoostClassifier(random_state=random_state, n_estimators=43, learning_rate=0.1, max_depth=1, class_weights=[0.9, 0.1], task_type='GPU')

###Проверка на тестовой выборке

In [ ]:
model.fit(x_train, y_train)
predictions = model.predict(x_test)
print("f1_score модели логистической регрессии:", f1_score(y_test, predictions))

0:	learn: 0.4523223	total: 7.63ms	remaining: 320ms
1:	learn: 0.3143743	total: 14.1ms	remaining: 289ms
2:	learn: 0.2177078	total: 20.5ms	remaining: 273ms
3:	learn: 0.1544075	total: 26.8ms	remaining: 261ms
4:	learn: 0.1158064	total: 34.7ms	remaining: 263ms
5:	learn: 0.0870922	total: 41ms	remaining: 253ms
6:	learn: 0.0680943	total: 47.5ms	remaining: 244ms
7:	learn: 0.0533028	total: 53.9ms	remaining: 236ms
8:	learn: 0.0442986	total: 60.5ms	remaining: 229ms
9:	learn: 0.0384534	total: 67.1ms	remaining: 221ms
10:	learn: 0.0332024	total: 73.5ms	remaining: 214ms
11:	learn: 0.0290424	total: 80.1ms	remaining: 207ms
12:	learn: 0.0265057	total: 86.6ms	remaining: 200ms
13:	learn: 0.0248466	total: 93.1ms	remaining: 193ms
14:	learn: 0.0230831	total: 99.5ms	remaining: 186ms
15:	learn: 0.0225214	total: 106ms	remaining: 179ms
16:	learn: 0.0211152	total: 112ms	remaining: 172ms
17:	learn: 0.0206994	total: 119ms	remaining: 165ms
18:	learn: 0.0199890	total: 126ms	remaining: 159ms
19:	learn: 0.0191818	total: 

<div class="alert alert-block alert-info">

**Итоговый вывод:** Таким образом модель градиентного бустинга, обученная на данных, преобразованных моделью BERT, показала лучший результат по метрике f1, чем модель логистической регрессии, обученная на данных, преобразованных техникой TfIdf.